<a href="https://colab.research.google.com/github/miniact/mangyaWAF/blob/master/mms_waf_modeling_ml_algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modeling and evaluation of machine learning model

Import all dependencies


In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
import pickle
from IPython.display import display
#evaluations
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve

In [2]:
data = pd.read_csv("https://trello-attachments.s3.amazonaws.com/5ed2d4107c349c221194b608/5ed2d453f0e5a45bcd8cf16c/9d9eab0bcf8afdfbdff5fdb173f78cd6/final-allpayload.csv",index_col="index")
data

,payload,is_malicious,injection_type,length,non-printable,punctuation,min-byte,max-byte,mean-byte,std-byte,distinct-byte,sql-keywords,js-keywords
index,,,,,,,,,,,,,
0,37662577P,0.0,LEGAL,9,0,0,50,80,56.333333,8.537499,6,0,0
1,shirting,0.0,LEGAL,8,0,0,103,116,109.000000,5.049752,7,0,1
2,&kw=%27;alert%28%27XSS%27%29;//,1.0,XSS,31,0,11,37,119,65.806452,26.617263,18,0,1
3,obeying,0.0,LEGAL,7,0,0,98,121,107.000000,7.151423,7,0,1
4,dictating,0.0,LEGAL,9,0,0,97,116,105.666667,6.599663,7,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110355,needlessness,0.0,LEGAL,12,0,0,100,115,107.666667,6.209312,5,1,0
110356,Mary-Jo,0.0,LEGAL,7,0,1,45,121,91.285714,25.257046,7,0,0
110357,marshaled,0.0,LEGAL,9,0,0,97,115,105.000000,6.463573,8,0,0


Selecting dependent and independent variables

In [5]:
Y = data['is_malicious']
"""independent_variables = data.columns
independent_variables = independent_variables.delete(1)
independent_variables"""
X = data.iloc[:,3:]
X

,length,non-printable,punctuation,min-byte,max-byte,mean-byte,std-byte,distinct-byte,sql-keywords,js-keywords
index,,,,,,,,,,
0,9,0,0,50,80,56.333333,8.537499,6,0,0
1,8,0,0,103,116,109.000000,5.049752,7,0,1
2,31,0,11,37,119,65.806452,26.617263,18,0,1
3,7,0,0,98,121,107.000000,7.151423,7,0,1
4,9,0,0,97,116,105.666667,6.599663,7,0,1
...,...,...,...,...,...,...,...,...,...,...
110355,12,0,0,100,115,107.666667,6.209312,5,1,0
110356,7,0,1,45,121,91.285714,25.257046,7,0,0
110357,9,0,0,97,115,105.000000,6.463573,8,0,0


# Modeling data with Xgboost Classifier

In [3]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [6]:
import xgboost as xgb
#?xgb.XGBClassifier()
xgb_classifer = xgb.XGBClassifier()
xgb_classifer.fit(X,Y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [7]:
%time xgb_classifer.fit(X,Y)

CPU times: total: 31.7 s
Wall time: 2.01 s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [8]:
data['predicted_is_malicious'] = xgb_classifer.predict(X)

In [9]:
data.head(30)

,payload,is_malicious,injection_type,length,non-printable,punctuation,min-byte,max-byte,mean-byte,std-byte,distinct-byte,sql-keywords,js-keywords,predicted_is_malicious
index,,,,,,,,,,,,,,
0,37662577P,0.0,LEGAL,9,0,0,50,80,56.333333,8.537499,6,0,0,0
1,shirting,0.0,LEGAL,8,0,0,103,116,109.000000,5.049752,7,0,1,0
2,&kw=%27;alert%28%27XSS%27%29;//,1.0,XSS,31,0,11,37,119,65.806452,26.617263,18,0,1,1
3,obeying,0.0,LEGAL,7,0,0,98,121,107.000000,7.151423,7,0,1,0
4,dictating,0.0,LEGAL,9,0,0,97,116,105.666667,6.599663,7,0,1,0
5,lafleur,0.0,LEGAL,7,0,0,97,117,106.714286,6.670067,6,0,0,0
6,capturers,0.0,LEGAL,9,0,0,97,117,109.444444,7.558823,8,0,0,0
7,8nca58z48,0.0,LEGAL,9,0,0,52,122,77.888889,26.904954,7,0,0,0
8,autocratic,0.0,LEGAL,10,0,0,97,117,107.100000,8.117266,7,0,0,0


In [10]:
data[["is_malicious","predicted_is_malicious"]]

,is_malicious,predicted_is_malicious
index,,
0,0.0,0
1,0.0,0
2,1.0,1
3,0.0,0
4,0.0,0
...,...,...
110355,0.0,0
110356,0.0,0
110357,0.0,0


# Storing the trained model into pickel file
to skip the training step all time and directly load trained data from file

In [11]:
#creating binary file

with open("mms_maf_final","wb") as file:
  pickle.dump(xgb_classifer,file)

In [12]:
#reading binary file

with open("mms_maf_final","rb") as file:
  xgb_classifer_pickle = pickle.load(file)

In [13]:
#testing the model via pickle file
data['predicted_is_malicious_pickle'] = xgb_classifer_pickle.predict(X)
data[["is_malicious","predicted_is_malicious","predicted_is_malicious_pickle"]]

,is_malicious,predicted_is_malicious,predicted_is_malicious_pickle
index,,,
0,0.0,0,0
1,0.0,0,0
2,1.0,1,1
3,0.0,0,0
4,0.0,0,0
...,...,...,...
110355,0.0,0,0
110356,0.0,0,0
110357,0.0,0,0


Storing model with joblib



In [15]:
import joblib
joblib.dump(xgb_classifer,"mms_waf_joblib")


['mms_waf_joblib']

In [16]:
#prwediction via joblib
#testing the model via pickle file
xgb_classifer_joblib = joblib.load("mms_waf_joblib")
data['predicted_is_malicious_joblib'] = xgb_classifer_joblib.predict(X)
data[["is_malicious","predicted_is_malicious","predicted_is_malicious_joblib"]]

,is_malicious,predicted_is_malicious,predicted_is_malicious_joblib
index,,,
0,0.0,0,0
1,0.0,0,0
2,1.0,1,1
3,0.0,0,0
4,0.0,0,0
...,...,...,...
110355,0.0,0,0
110356,0.0,0,0
110357,0.0,0,0


# Integration with website


In [17]:
independent_variables=['length', 'non-printable','punctuation', 'min-byte', 'max-byte', 'mean-byte', 'std-byte','distinct-byte', 'sql-keywords', 'js-keywords']
independent_variables

['length',
 'non-printable',
 'punctuation',
 'min-byte',
 'max-byte',
 'mean-byte',
 'std-byte',
 'distinct-byte',
 'sql-keywords',
 'js-keywords']

In [18]:
independent_variables=data.columns

sql_keywords = pd.read_csv('https://trello-attachments.s3.amazonaws.com/5ed2d4107c349c221194b608/5ed2d453f0e5a45bcd8cf16c/435e639346787ce2b495a16e9f690ef5/SQLKeywords.txt', index_col=False)
js_keywords = pd.read_csv("https://trello-attachments.s3.amazonaws.com/5ed2d4107c349c221194b608/5ed2d453f0e5a45bcd8cf16c/dedc7eb9846a30c252cd950a0e2153d9/JavascriptKeywords.txt",index_col=False)
def calculate_features_and_predict(payload):
  features = {} 
  payload = str(payload)
  features['length'] = len(payload)
  features['non-printable'] = len([1 for letter in payload if letter not in string.printable])
  features['punctuation'] = len([1 for letter in payload if letter in string.punctuation])
  features['min-byte'] = min(bytearray(payload,'utf-8'))
  features['max-byte'] = max(bytearray(payload,'utf-8'))
  features['mean-byte'] = np.mean(bytearray(payload,'utf-8'))
  features['std-byte'] = np.std(bytearray(payload,'utf-8'))
  features['distinct-byte'] = len(set(bytearray(payload,'utf-8')))
  features['sql-keywords'] = len([1 for keyword in sql_keywords['Keyword'] if str(keyword).lower() in payload.lower()])
  features['js-keywords'] = len([1 for keyword in js_keywords['Keyword'] if str(keyword).lower() in payload.lower()])
  #payload_df = pd.DataFrame(data=features,index=[0],columns=independent_variables)
  payload_df = pd.DataFrame(features,index=[0])
  display(payload_df)
  result = xgb_classifer.predict(payload_df)
  #display(result)
  return result[0]


In [19]:
calculate_features_and_predict("<>")

,length,non-printable,punctuation,min-byte,max-byte,mean-byte,std-byte,distinct-byte,sql-keywords,js-keywords
0,2,0,2,60,62,61.0,1.0,2,0,0


0

In [20]:
payload=''
while (payload != 'exit' ):
  payload = input("Enter payload")
  result = calculate_features_and_predict(payload)
  if(result > 0):
    print(f"Your payload {payload} is malicious - 403 error\n")
  else:
    print(f"Your payload {payload} is safe 200 OK\n")

Enter payloadhello


,length,non-printable,punctuation,min-byte,max-byte,mean-byte,std-byte,distinct-byte,sql-keywords,js-keywords
0,5,0,0,101,111,106.4,3.498571,4,0,0


Your payload hello is safe 200 OK

Enter payload1==1


,length,non-printable,punctuation,min-byte,max-byte,mean-byte,std-byte,distinct-byte,sql-keywords,js-keywords
0,4,0,2,49,61,55.0,6.0,2,0,0


Your payload 1==1 is safe 200 OK

Enter payload<script>alert(1)</script>


...

Your payload <script>alert(1)</script> is malicious - 403 error

Enter payload'1==1'


,length,non-printable,punctuation,min-byte,max-byte,mean-byte,std-byte,distinct-byte,sql-keywords,js-keywords
0,6,0,4,39,61,49.666667,8.993825,3,0,0


Your payload '1==1' is malicious - 403 error

Enter payloadexit


,length,non-printable,punctuation,min-byte,max-byte,mean-byte,std-byte,distinct-byte,sql-keywords,js-keywords
0,4,0,0,101,120,110.5,7.762087,4,0,0


Your payload exit is safe 200 OK

